In [142]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [143]:
pt_low, pt_high = 1, 16
phi_v_low, phi_v_high = 1,42
itype = 4
do_scale_folder = 1
lefty =  [1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4]
righty = [2.9,2.5,2.3,2.2,2.1,1.9,1.9,1.9,1.9,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8]

In [144]:
isigma = 2
N_centr = 6
N_options = 1
N_vtx_bins = 4
centralities = ["0-93%","0-20%","20-40%","40-60%","60-93%","40-93%"]
centrality = [0,9,0,1,2,3,4,5,6,9,4,9]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_name="m_ee_Run14AuAu_25th_test_19197_1000runs.root"
#m_ee_Run14AuAu_25th_test_19197_350runs
#m_ee_Run14AuAu_23rd_test_19184_500runs m_ee_Run14AuAu_24th_test_19192_980runs
#m_ee_Run14AuAu_21th_test_19127_970runs  m_ee_Run14AuAu_22th_test_19180_1078runs
# m_ee_Run14AuAu_19th_test_19117_1016runs m_ee_Run14AuAu_20th_test_19121_999runs
#m_ee_Run14AuAu_17th_test_19114_790runs m_ee_Run14AuAu_18th_test_19115_1073runs
#m_ee_Run14AuAu_15th_test_19107_1080runs m_ee_Run14AuAu_16th_test_19109_1078runs
#m_ee_Run14AuAu_bakers_dozen_test_19104_900runs m_ee_Run14AuAu_14th_test_19105_780runs
#m_ee_Run14AuAu_eleventh_test_19100_800runs m_ee_Run14AuAu_twelfth_test_19102_920runs
#"m_ee_Run14AuAu_eighth_test_19090_900runs.root" m_ee_Run14AuAu_tenth_test_19097_800runs
#"m_ee_Run14AuAU_fourth_test_19072_800runs.root"#"m_ee_Run14AuAU_first_test_19060_100runs.root"
#m_ee_Run14AuAu_ninth_test_19090_540runs

In [145]:
hists_ep_FG_read, hists_ep_BG_read, hists_ee_FG_read, hists_ee_BG_read = [], [], [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_name, "read")

for iOpt in range(0,N_options):
    histo_name = [f"inv_mass_ee_DCA_V{itype}_",f"inv_mass_ee_diff_arm_V{itype}_"]
    histo_name_like = [f"delt_phi_ee_DCA_V{itype}_",f"delt_phi_ee_diff_arm_V{itype}_"]
    print(histo_name)
    hists_ep_FG_read_centr, hists_ep_BG_read_centr, hists_ee_FG_read_centr, hists_ee_BG_read_centr = [], [], [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        hist_FG = infile.Get("c00_z00_r00/"+histo_name[0]+"FG12")
        hist_FG.Add(hist_FG,-1)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get("c00_z00_r00/"+histo_name[0]+"BG12")
        hist_BG.Add(hist_BG,-1)
        hist_BG.SetDirectory(root.nullptr)
        hist_FG_like = infile.Get("c00_z00_r00/"+histo_name_like[0]+"FG11")
        hist_FG_like.Add(hist_FG_like,-1)
        hist_FG_like.SetDirectory(root.nullptr)
        hist_BG_like = infile.Get("c00_z00_r00/"+histo_name_like[0]+"BG11")
        hist_BG_like.Add(hist_BG_like,-1)
        hist_BG_like.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(N_vtx_bins):
                for k in range(0,1):
                    dirname=f"c0{i}_z0{j}_r00/"
                    if i>9: dirname=f"c{i}_z0{j}_r00/"
                    hist_FG_local = infile.Get(dirname+histo_name[k]+"FG12")
                    hist_BG_local = infile.Get(dirname+histo_name[k]+"BG12")
                    hist_FG_like_local = infile.Get(dirname+histo_name_like[k]+"FG11")
                    hist_BG_like_local = infile.Get(dirname+histo_name_like[k]+"BG11")
                    hist_FG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"FG22"))
                    hist_BG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"BG22"))
                    eta_FG_porj = hist_FG_like_local.ProjectionX(f"FG_loc_{icent}_{i*20+j}",phi_v_low,phi_v_high,pt_low,pt_high)
                    eta_BG_porj = hist_BG_like_local.ProjectionX(f"BG_loc_{icent}_{i*20+j}",phi_v_low,phi_v_high,pt_low,pt_high)

                    left = eta_FG_porj.FindBin(1.0)
                    right = eta_FG_porj.FindBin(1.99)+1

                    scale = eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1)
                    if not do_scale_folder: scale = 1
                    
                    hist_FG.Add(hist_FG_local)
                    hist_BG.Add(hist_BG_local,scale)
                    hist_FG_like.Add(hist_FG_like_local)
                    hist_BG_like.Add(hist_BG_like_local,scale)
                    hist_stat.Add(infile.Get(dirname+"PoolStatistics"))

        hists_ep_FG_read_centr.append(hist_FG)
        hists_ep_BG_read_centr.append(hist_BG)
        hists_ee_FG_read_centr.append(hist_FG_like)
        hists_ee_BG_read_centr.append(hist_BG_like)
        hist_stat_centr.append(hist_stat)
    hists_ep_FG_read.append(hists_ep_FG_read_centr)
    hists_ep_BG_read.append(hists_ep_BG_read_centr)
    hists_ee_FG_read.append(hists_ee_FG_read_centr)
    hists_ee_BG_read.append(hists_ee_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)

infile.Close()

['inv_mass_ee_DCA_V4_', 'inv_mass_ee_diff_arm_V4_']


In [146]:
iOption = 0
rebin = 1
new_pt_array = array( 'd', [0.,0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])
N_new_bins = len(new_pt_array)-1
print(N_new_bins)

35


In [147]:

c3 = root.TCanvas(f"c2",f"c2",1600,900)
c3.Divide(3,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ee_FG_read[iOption][icentr].ProjectionX(f"px{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"px_BG_{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinContent(ibin,proj_BG.GetBinContent(ibin)/proj_BG.GetBinWidth(ibin))
    proj.Scale(1./hist_stat_read[iOption][icentr].GetBinContent(2))
    proj_BG.Scale(1./hist_stat_read[iOption][icentr].GetBinContent(2))
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    root.gPad.SetLogy(1)
    proj_BG.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
c3.Print("output/m_ee/scale_BG.png")

In [148]:
c5 = root.TCanvas("ccc33","ccc33",1500,1200)
iOpt = 0
pt_bins=[1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12,13,13,14,14,15,15,16,16]
c5.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c5.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionX(f"px{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionX(f"px_BG_{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG.Add(proj_BG,-1)

    for ipt in range(int(len(pt_bins)/2)):
        proj2 = hists_ee_FG_read[iOption][icentr].ProjectionX(f"pxl{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        proj2_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"pxl_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        left = proj2.FindBin(lefty[ipt])
        right = proj2.FindBin(righty[ipt])+1
        scale = proj2.Integral(left,right)/(proj2_BG.Integral(left,right)+1)
        #if abs(scale - 1) > 0.1: print(icentr, ipt, scale)
        proj_BG.Add(hists_ep_BG_read[iOption][icentr].ProjectionX(f"pxll_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1]),scale)
    
    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        proj_BG.SetBinError(ibin,(proj_BG.GetBinContent(ibin))**0.5)
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinContent(ibin,proj_BG.GetBinContent(ibin)/proj_BG.GetBinWidth(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,proj_BG.GetBinError(ibin)/proj_BG.GetBinWidth(ibin))
    
    proj.Scale(1./hist_stat_read[iOption][icentr].GetBinContent(2))
    proj_BG.Scale(1./hist_stat_read[iOption][icentr].GetBinContent(2))
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,0.9)
    Format_Graph(proj_BG,49,1.5,2,3,2)
    root.gPad.SetLogy(1)
    proj.Draw("p")
    proj_BG.Draw("same p")
    #proj.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    legends[-1].AddEntry(proj,"FG","pl")
    legends[-1].AddEntry(proj_BG,"BG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/m_ee/m_ee_newpt_FG_BG.png")

In [149]:
c3 = root.TCanvas(f"c2",f"c2",1500,1200)
sub_koeff=[1.0,1,0.99,0.98,1.0,1.0]
sub_koeff=[0.95,0.93,0.9,0.7,0.4,0.7]
pt_bins=[1,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12,13,13,14,14,15,15,16,16]
c3.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionX(f"px{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionX(f"px_BG_{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG.Add(proj_BG,-1)

    for ipt in range(int(len(pt_bins)/2)):
        proj2 = hists_ee_FG_read[iOption][icentr].ProjectionX(f"pxl{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        proj2_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"pxl_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        left = proj2.FindBin(lefty[ipt])
        right = proj2.FindBin(righty[ipt])+1
        scale = proj2.Integral(left,right)/(proj2_BG.Integral(left,right)+1)
        #if abs(scale - 1) > 0.1: print(icentr, ipt, scale)
        proj_BG.Add(hists_ep_BG_read[iOption][icentr].ProjectionX(f"pxll_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1]),scale)
    
        
    proj_BG.Scale(1)
    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)-sub_koeff[icentr]*proj_BG.GetBinContent(ibin))
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        
    proj.Scale(1./hist_stat_read[iOption][icentr].GetBinContent(2))
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1,4,3,4,0.85,0.9)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetLineWidth(3)
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].AddEntry(proj,"FG-BG","pl")
    legends[-1].Draw()

    print(round(hist_stat_read[iOption][icentr].GetBinContent(2)/1e9,3),"*10^9")
    

c3.Draw()
c3.Print("output/m_ee/new_m_ee_newpt_FG.png")

10.928 *10^9
2.461 *10^9
2.416 *10^9
2.382 *10^9
3.669 *10^9
6.051 *10^9


In [150]:
#!jupyter nbconvert --to html m_ee_plotter.ipynb --HTMLExporter.theme=dark;

In [151]:
c3 = root.TCanvas(f"c2",f"c2",1200,800)
sub_koeff=[1.0,1.0,1.0,1.0,0.0,1.0]
sub_koeff=[0.959,0.95,0.91,0.83,0.8,0.89]
pt_bins=[1,7,8,16]
c3.Divide(1,1)
legends =[]
projs,projs_BG=[],[]
hists_ep_FG_read[iOption][0].Add(hists_ep_FG_read[iOption][0],-1)
hists_ep_BG_read[iOption][0].Add(hists_ep_BG_read[iOption][0],-1)
hists_ee_FG_read[iOption][0].Add(hists_ee_FG_read[iOption][0],-1)
hists_ee_BG_read[iOption][0].Add(hists_ee_BG_read[iOption][0],-1)
for ictr in range(1,5):
    hists_ep_FG_read[iOption][0].Add(hists_ep_FG_read[iOption][ictr])
    hists_ep_BG_read[iOption][0].Add(hists_ep_BG_read[iOption][ictr])
    hists_ee_FG_read[iOption][0].Add(hists_ee_FG_read[iOption][ictr])
    hists_ee_BG_read[iOption][0].Add(hists_ee_BG_read[iOption][ictr])


for icentr in range(1):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionX(f"px{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionX(f"px_BG_{icentr}{iOption}",phi_v_low,phi_v_high,pt_low,pt_high)
    proj_BG.Add(proj_BG,-1)

    for ipt in range(int(len(pt_bins)/2)):
        proj2 = hists_ee_FG_read[iOption][icentr].ProjectionX(f"pxl{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        proj2_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"pxl_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1])
        left = proj2.FindBin(lefty[ipt])
        right = proj2.FindBin(righty[ipt])+1
        scale = proj2.Integral(left,right)/(proj2_BG.Integral(left,right)+1)
        #if abs(scale - 1) > 0.1: print(icentr, ipt, scale
        proj_BG.Add(hists_ep_BG_read[iOption][icentr].ProjectionX(f"pxll_BG_{icentr}{ipt}",phi_v_low,phi_v_high,pt_bins[2*ipt],pt_bins[2*ipt+1]),scale)
    
        
    proj_BG.Scale(1)
    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)-sub_koeff[icentr]*proj_BG.GetBinContent(ibin))
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        
    proj.Scale(1./hist_stat_read[iOption][icentr].GetBinContent(2))
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1,4,3,4,0.85,0.9)
    root.gPad.SetLogy(1)
    proj.SetMinimum(1e-10)
    proj.Draw("")
    proj.SetLineWidth(3)
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].AddEntry(proj,"FG-BG","pl")
    legends[-1].Draw()

    print(round(hist_stat_read[iOption][icentr].GetBinContent(2)/1e9,3),"*10^9")
    

c3.Draw()
c3.Print("output/m_ee/m_ee_newpt_FG.png")

10.928 *10^9
